In [13]:
import random
import numpy as np
import networkx as nx  # Ensure NetworkX is installed

def approx_mfvs_ga(G, population_size=10, generations=100, mutation_rate=0.1, elite_size=2):
    """
    Approximates the Minimum Feedback Vertex Set (MFVS) in a directed graph using a genetic algorithm with elitism.

    Args:
        G: A NetworkX directed graph.
        population_size: Size of the population in each generation (default: 10).
        generations: Number of generations to evolve (default: 100).
        mutation_rate: Probability of mutating a bit in each chromosome (default: 0.1).
        elite_size: Number of top individuals to carry over to the next generation without changes (default: 2).

    Returns:
        A set of nodes representing an approximation to the MFVS.
    """

    # Define fitness function
    def fitness(chromosome):
        mfvs = set(node for node, gene in enumerate(chromosome) if gene == 1)
        for u, v in G.edges():
            if u in mfvs or v in mfvs:
                continue
            return -1  # Penalize invalid solutions
        return -len(mfvs)  # We want to minimize the MFVS size

    # Initialize population
    population = [[random.randint(0, 1) for _ in range(len(G))] for _ in range(population_size)]

    # Selection function: Roulette Wheel Selection
    def roulette_wheel_selection(population, fitness_scores):
        total_fitness = sum(fitness_scores)
        probabilities = [f / total_fitness for f in fitness_scores]
        return random.choices(population, weights=probabilities, k=1)[0]

    # Crossover function: Single-Point Crossover
    def single_point_crossover(parent1, parent2):
        crossover_point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2

    for _ in range(generations):
        # Evaluate fitness
        fitness_scores = [fitness(chromosome) for chromosome in population]

        # Sort population by fitness score in ascending order because we minimize
        sorted_indices = np.argsort(fitness_scores)
        sorted_population = [population[i] for i in sorted_indices]

        # Elitism: Directly carry over the top `elite_size` individuals
        new_population = sorted_population[:elite_size]

        # Fill the rest of the population with offspring
        while len(new_population) < population_size:
            parent1 = roulette_wheel_selection(population, fitness_scores)
            parent2 = roulette_wheel_selection(population, fitness_scores)
            child1, child2 = single_point_crossover(parent1, parent2)

            # Mutation
            for chromosome in [child1, child2]:
                for i in range(len(chromosome)):
                    if random.random() < mutation_rate:
                        chromosome[i] = 1 - chromosome[i]

            new_population.extend([child1, child2])

        # Ensure the population size is correct
        population = new_population[:population_size]

    # Find best chromosome in the final population
    best_fitness = max([fitness(chromosome) for chromosome in population])
    best_chromosome = population[[fitness(chromosome) for chromosome in population].index(best_fitness)]
    return set(i for i, gene in enumerate(best_chromosome) if gene == 1)

# Example usage
if __name__ == "__main__":
    # Create a sample directed graph
    G = nx.DiGraph()
    G.add_edges_from([('A', 1), (1, 2), (2, 3), (3, 4), (4, 1)])
    
    # Run the genetic algorithm
    mfvs = approx_mfvs_ga(G)
    print("Approximate Minimum Feedback Vertex Set:", mfvs)


Approximate Minimum Feedback Vertex Set: {0, 1, 4}


In [15]:
import random
import numpy as np
import networkx as nx  # Make sure NetworkX is installed

def approx_mfvs_ga(G, initial_guess=None, population_size=10, generations=100, mutation_rate=0.1, elite_size=2):
    """
    Approximates the Minimum Feedback Vertex Set (MFVS) in a directed graph using a genetic algorithm with elitism.

    Args:
        G: A NetworkX directed graph.
        initial_guess: A set of node names representing an initial guess for the MFVS (default: None).
        population_size: Size of the population in each generation (default: 10).
        generations: Number of generations to evolve (default: 100).
        mutation_rate: Probability of mutating a bit in each chromosome (default: 0.1).
        elite_size: Number of top individuals to carry over to the next generation without changes (default: 2).

    Returns:
        A set of node names representing an approximation to the MFVS.
    """

    # Map initial guess to chromosome
    def guess_to_chromosome(guess):
        chromosome = [0] * len(G)
        node_list = list(G.nodes())
        for node in guess:
            if node in node_list:
                chromosome[node_list.index(node)] = 1
        return chromosome

    # Define fitness function
    def fitness(chromosome):
        mfvs = set(node for node, gene in enumerate(chromosome) if gene == 1)
        for u, v in G.edges():
            if u in mfvs or v in mfvs:
                continue
            return -1  # Penalize invalid solutions
        return -len(mfvs)  # We want to minimize the MFVS size

    # Initialize population
    population = [guess_to_chromosome(initial_guess)] if initial_guess else []
    while len(population) < population_size:
        population.append([random.randint(0, 1) for _ in range(len(G))])

    # Selection function: Roulette Wheel Selection
    def roulette_wheel_selection(population, fitness_scores):
        total_fitness = sum(fitness_scores)
        probabilities = [f / total_fitness for f in fitness_scores]
        return random.choices(population, weights=probabilities, k=1)[0]

    # Crossover function: Single-Point Crossover
    def single_point_crossover(parent1, parent2):
        crossover_point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2

    for _ in range(generations):
        # Evaluate fitness
        fitness_scores = [fitness(chromosome) for chromosome in population]

        # Sort population by fitness score in ascending order because we minimize
        sorted_indices = np.argsort(fitness_scores)
        sorted_population = [population[i] for i in sorted_indices]

        # Elitism: Directly carry over the top `elite_size` individuals
        new_population = sorted_population[:elite_size]

        # Fill the rest of the population with offspring
        while len(new_population) < population_size:
            parent1 = roulette_wheel_selection(population, fitness_scores)
            parent2 = roulette_wheel_selection(population, fitness_scores)
            child1, child2 = single_point_crossover(parent1, parent2)

            # Mutation
            for chromosome in [child1, child2]:
                for i in range(len(chromosome)):
                    if random.random() < mutation_rate:
                        chromosome[i] = 1 - chromosome[i]

            new_population.extend([child1, child2])

        # Ensure the population size is correct
        population = new_population[:population_size]

    # Find best chromosome in the final population
    best_fitness = max([fitness(chromosome) for chromosome in population])
    best_chromosome = population[[fitness(chromosome) for chromosome in population].index(best_fitness)]
    
    # Convert indices to node names
    node_list = list(G.nodes())
    return set(node_list[i] for i, gene in enumerate(best_chromosome) if gene == 1)

# Example usage
if __name__ == "__main__":
    # Create a sample directed graph with named nodes
    G = nx.DiGraph()
    G.add_edges_from([("A", "B"), ("B", "C"), ("C", "D"), ("D", "E"), ("E", "B")])
    
    # Initial guess for the MFVS
    initial_guess = {"B", "D"}
    
    # Run the genetic algorithm with the initial guess
    mfvs = approx_mfvs_ga(G, initial_guess)
    print("Approximate Minimum Feedback Vertex Set:", mfvs)


Approximate Minimum Feedback Vertex Set: {'B', 'D'}


In [16]:
import networkx as nx
import random
import math

def simulated_annealing(G, initial_temp, cooling_rate, termination_temp):
    current_solution = random_solution(G)
    current_cost = evaluate_solution(G, current_solution)
    temperature = initial_temp

    while temperature > termination_temp:
        neighbor_solution = generate_neighbor(current_solution)
        neighbor_cost = evaluate_solution(G, neighbor_solution)
        cost_diff = neighbor_cost - current_cost

        if cost_diff < 0 or math.exp(-cost_diff / temperature) > random.random():
            current_solution = neighbor_solution
            current_cost = neighbor_cost

        temperature *= cooling_rate

    return current_solution

def random_solution(G):
    # Generate a random initial solution
    return [random.choice([0, 1]) for _ in range(len(G))]

def evaluate_solution(G, solution):
    # Evaluate the solution: size of the set - penalties for cycles
    # Implement cycle detection and penalty calculation based on the solution
    return len([x for x in solution if x == 1])  # Simplified

def generate_neighbor(solution):
    # Generate a neighbor solution
    index = random.randint(0, len(solution) - 1)
    new_solution = solution.copy()
    new_solution[index] = 1 - new_solution[index]  # Flip the state
    return new_solution

# Example usage
G = nx.DiGraph([(0, 1), (1, 2), (2, 3), (3, 1)])  # Example graph
initial_temp = 100
cooling_rate = 0.95
termination_temp = 1

mfvs = simulated_annealing(G, initial_temp, cooling_rate, termination_temp)
print("Approximate MFVS:", mfvs)


Approximate MFVS: [1, 0, 0, 0]
